# Lending Club Analysis

 - Part 1: Pre-processing
 - Part 2: Feature engineering (spelling errors)
 - Part 3: Predict default

## Part 1: Pre-processing

In [64]:
import pandas as pd
import os
import re
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

df = pd.read_csv("LoanStats3a.csv", skiprows=[0])
# Clean the preface of "borrower added on xx date"
df['desc_clean'] = df['desc'].str.replace(r"Borrower added on \d+\/\d+\/\d+ > ", "")

# Remove HTML tags
df['desc_clean'] = [re.sub(r"<[a-z]+>", "", str(desc)) for desc in df['desc_clean']]
df['desc_clean'] = [desc.replace("<br/>", " ").strip() for desc in df['desc_clean']]

# Remove email addresses and webpages
clean_descs = []
for i, desc in enumerate(tqdm(df['desc_clean'])):
    # Go back to the text we originally had taken typos from        
    n = 0
    desc = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}", 'e-mail', desc) #Email addresses
    desc = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", 'webpage', desc) #webpages
    desc = re.sub(r"([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:s~+#-]*[\w@?^=%&/~+#-])?", 'webpage', desc) #webpages
    clean_descs.append(desc)
df['desc_clean'] = clean_descs

# remove columns with all NA
df = df.dropna(how = 'all', axis='columns')

print("The Lending Club data set from 2007-2011 have free-form text description, with total number of loans: ", len(df['desc_clean']))


100%|██████████| 42538/42538 [00:02<00:00, 16049.59it/s]


The Lending Club data set from 2007-2011 have free-form text description, with total number of loans:  42538


In [65]:
# Charged off: the loan has been in Default for 30 days or more and no more future payments are expected
#"Does notemeet credit policy" is irrelevant: https://forum.lendacademy.com/?topic=2427.msg20813#msg20813
df.loc[df['loan_status'] == "Does not meet the credit policy. Status:Fully Paid", 'loan_status'] = 'Fully Paid'
df.loc[df['loan_status'] == "Does not meet the credit policy. Status:Charged Off", 'loan_status'] = 'Charged Off'

n = df['loan_status'].value_counts()
print("Default rate is ", np.round(n[1]/(n[1]+n[0])*100, 2), "%")

Default rate is  15.12 %


In [66]:
# Get the number of words in a description
df['words_desc'] = df['desc_clean'].str.split().str.len()
df['has_words'] = np.where(df['words_desc'] > 0, 1, 0)
df['has_words'].value_counts()
n = df['has_words'].value_counts()
print("% of loan applications with a description is ", np.round(n[1]/(n[1]+n[0])*100, 2), "%")

# compare default for those with and those without a description
sub = df.loc[df['words_desc'] > 0]
n = sub['loan_status'].value_counts()
print("Default rate for those with a description is ", np.round(n[1]/(n[1]+n[0])*100, 2), "%")

sub = df.loc[df['words_desc'] == 0]
n = sub['loan_status'].value_counts()
print("Default rate for those without a description is ", np.round(n[1]/(n[1]+n[0])*100, 2), "%")

% of loan applications with a description is  99.47 %
Default rate for those with a description is  15.13 %
Default rate for those without a description is  12.44 %


In [67]:
# For now, remove those without free-form comments
df = df.loc[df['desc_clean'].notnull()]
df = df.loc[df['words_desc'] > 0]

# found one entry in Spanish - remove
df = df[df['desc_clean'] != "Hola, gracias por el prestamo, no les voy a fallar en ningun pago. Espero que los 206,97 USD sigan sin aumentar hasta terminar mi pago total de 36 cuotas de 206,97 USD, thank you."] # Remove this one entry

# Get length
df['len_desc'] = df['desc_clean'].str.len()

df = df[~df['loan_status'].isnull()] #3 null loan status
df['loan_status'].value_counts()

Fully Paid     35906
Charged Off     6403
Name: loan_status, dtype: int64

In [68]:
temp = df.groupby('loan_status')['words_desc', 'len_desc'].describe().reset_index()
temp.columns = [' '.join(col).strip() for col in temp.columns.values]
temp = temp.drop(["len_desc count", "words_desc 25%","words_desc 75%","words_desc min", "words_desc max","len_desc 25%","len_desc 75%","len_desc min","len_desc max"], axis = 1)
temp.columns = ['Loan Status', 'N', 'Mean: # words', 'Std dev: # words', 'Median: # words', 'Mean: # characters', 'Std dev: # characters', 'Median: # characters']
temp = np.round(temp, decimals=1)
temp['N'] = temp['N'].astype('int')
temp['N'] = temp['N'].apply('{:,}'.format)
temp.index = temp['Loan Status']
temp = temp.drop('Loan Status', axis = 1)
latex_output = temp.to_latex()
with open("summary_stats_length.txt", "w") as text_file:
    text_file.write(latex_output)

pd.options.display.float_format = '{:,}'.format

temp = df.groupby('loan_status')['loan_amnt', 'annual_inc'].describe().reset_index()
temp.columns = [' '.join(col).strip() for col in temp.columns.values]
temp = temp.drop(["annual_inc count", "loan_amnt 25%","loan_amnt 75%","loan_amnt min", "loan_amnt max","annual_inc 25%","annual_inc 75%","annual_inc min","annual_inc max"], axis = 1)
temp.columns = ['Loan Status', 'N', 'Mean: Loan Amount', 'Std dev: Loan Amount', 'Median: Loan Amount', 'Mean: Income', 'Std dev: Income', 'Median: Income']
temp = np.round(temp, decimals=1)
temp.index = temp['Loan Status']
temp = temp.drop('Loan Status', axis = 1)
temp['N'] = temp['N'].astype('int')
temp['N'] = temp['N'].apply('{:,}'.format)

latex_output = temp.to_latex()
with open("summary_stats_loan_income.txt", "w") as text_file:
    text_file.write(latex_output)

In [69]:
# reset IDs
df['id'] = range(0, df.shape[0])
# Remove columns about outcome (e.g. interest rate offered, grade, lateness, recoveries, debt settlements, etc.)
keep = ['id', 'loan_amnt', 'annual_inc', 'emp_length', 'home_ownership', 'loan_status', 'desc', 'purpose', 'zip_code', 'addr_state', 'desc_clean', 'words_desc', 'has_words', 'len_desc']
df = df[keep]

In [70]:
import statsmodels.api as sm

from patsy import dmatrices
df['loan_status'] = df['loan_status'].map({'Charged Off':1, 'Fully Paid':0})
# Split into features and labels
y, X = dmatrices("loan_status ~ loan_amnt + annual_inc", data=df, return_type='dataframe') #+ C(home_ownership,Treatment(reference='RENT')) + C(purpose,Treatment(reference='debt_consolidation'))
X.columns = ["Intercept", "Loan amount", "Annual income"] #"Home ownership: mortgage", "Home ownership: none", "Home ownership: other","Home ownership: own", "Purpose: car", "Purpose: credit card", "Purpose: educational", "Purpose: home improvement", "Purpose: house", "Purpose: major purchase", "Purpose: medical", "Purpose: moving", "Purpose: other", "Purpose: renewable energy", "Purpose: small business", "Purpose: vacation", "Purpose: wedding"
# View the summary
logit = sm.Logit(y, X)
results = logit.fit()
results.summary2()

Optimization terminated successfully.
         Current function value: 0.421451
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.008     
Dependent Variable: loan_status      AIC:              35664.9833
Date:               2020-04-22 11:30 BIC:              35690.9413
No. Observations:   42305            Log-Likelihood:   -17829.   
Df Model:           2                LL-Null:          -17982.   
Df Residuals:       42302            LLR p-value:      7.7995e-67
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-----------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept        -1.6734   0.0297 -56.3143 0.0000 -1.7316 -1.6151
Loan amount       0.0000   0.0000  14.3088 0.0000  0.0000  0.0000
Annual income    -0.0000   0.0000 -13.5691 0.0000 -0.0000 -0.0000
=================================================================

"""

In [71]:
latex_output = results.summary2().as_latex()
with open("00_base_model_Logit_LaTeX.txt", "w") as text_file:
    text_file.write(latex_output)

In [72]:
from math import e
median_income = df['annual_inc'].median()
print("Median income: " + str(median_income))
median_loan = df['loan_amnt'].median()
print("Median loan: " + str(median_loan))

# Function to return the probability, computes e^x/(1+e^x)
coefs = results.params.values
coefs

print("If you have median income and median loan for debt consolidation and rent your home, your probability of default is:")
e_value = e**(coefs[0]+(coefs[1]*median_loan)+(coefs[2]*median_income))

print(np.round(e_value/(1+e_value)*100, 2), "%")

Median income: 59000.0
Median loan: 9750.0
If you have median income and median loan for debt consolidation and rent your home, your probability of default is:
15.05 %


## Part 2: Feature engineering (spelling errors)

### Identify spelling mistakes using Stanford NER

In [73]:
'''
ONLY TO RUN FOR THE FIRST TIME - SKIP IF DATA SAVED

import enchant
d = enchant.Dict("en_US")
from StanfordNER import *

# Uses StanfordNER code to find named entities
set(list(Organisation(NER('MasterCard, LendingClub, Citi, REO, APR, PNC, HIPPA, wna are some examples of named entities'))))

from tqdm import tqdm
n_typos = []
typos = []
clean_descs = []
typos_list = []
rec = []

# Look for typos
for i, desc in enumerate(tqdm(df['desc_clean'])):
    n = 0
    desc = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}", '', desc) #Email addresses
    desc = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", '', desc) #webpages
    desc = re.sub(r"([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", '', desc) #webpages
    desc = re.sub(r"[^a-zA-Z' ]+", ' ', desc)
    typos_tmp = []
    #orgs = set(list(Organisation(NER(desc))))
    new_desc = [word for word in desc.split()]# if word not in orgs]
    desc_clean = ''
    for word in desc.split(): #in new_desc for org replacement
        if d.check(word) == False & d.check(word.upper()) == False:
            n += 1
            typos.append(word)
            if len(d.suggest(word)) > 0:
                rec.append(d.suggest(word)[0])
            else:
                rec.append("")
            typos_tmp.append(word)
        else:
            desc_clean += word
            desc_clean += " "
    typos_list.append(typos_tmp)
    clean_descs.append(desc_clean)
    n_typos.append(n)

df['typos'] = typos_list

typ = pd.DataFrame({'recommended': rec, 'typo': typos})
typ.drop_duplicates()

typ.to_csv("typos_v02.csv", index = False)
'''

'\nONLY TO RUN FOR THE FIRST TIME - SKIP IF DATA SAVED\n\nimport enchant\nd = enchant.Dict("en_US")\nfrom StanfordNER import *\n\n# Uses StanfordNER code to find named entities\nset(list(Organisation(NER(\'MasterCard, LendingClub, Citi, REO, APR, PNC, HIPPA, wna are some examples of named entities\'))))\n\nfrom tqdm import tqdm\nn_typos = []\ntypos = []\nclean_descs = []\ntypos_list = []\nrec = []\n\n# Look for typos\nfor i, desc in enumerate(tqdm(df[\'desc_clean\'])):\n    n = 0\n    desc = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,4}", \'\', desc) #Email addresses\n    desc = re.sub(r"(http|ftp|https)://([\\w_-]+(?:(?:\\.[\\w_-]+)+))([\\w.,@?^=%&:/~+#-]*[\\w@?^=%&/~+#-])?", \'\', desc) #webpages\n    desc = re.sub(r"([\\w_-]+(?:(?:\\.[\\w_-]+)+))([\\w.,@?^=%&:/~+#-]*[\\w@?^=%&/~+#-])?", \'\', desc) #webpages\n    desc = re.sub(r"[^a-zA-Z\' ]+", \' \', desc)\n    typos_tmp = []\n    #orgs = set(list(Organisation(NER(desc))))\n    new_desc = [word for word in desc.split()]

In [74]:
typ = pd.read_csv("typos_v02.csv")
columns = ['typo', 'recommended']
typ = typ.drop_duplicates()
typ = typ[columns]
typ.head()

,typo,recommended
0,im,mi
2,takingmeplaces,marketplaces
3,payor,mayor
4,Downpayment,Down payment
5,eg,g


** SKIP IF ALREADY SAVED **

In [ ]:
'''
from StanfordNER import *
from tqdm import tqdm
orgs = []
for i, typo in enumerate(tqdm(typ['typo'])):
    if len(Organisation(NER(typo))) > 0:
        orgs.append(typo)

orgs[:10]

orgs_df = pd.DataFrame({"organisations": orgs})
orgs_df.to_csv("orgs.csv")
'''

In [75]:
orgs_df = pd.read_csv("orgs.csv")

orgs = orgs_df['organisations']
orgs[:10]
orgs = list(orgs)

len_0 = typ.shape[0]
print("Unique typos: " + str(len_0))
typ = typ[~typ['typo'].isin(orgs)]
len_1 = typ.shape[0]
diff = len_0 - len_1
print("Organisations removed: " + str(diff))
len_0 = len_1

lowercase_orgs = typ[typ['typo'].isin([org.lower() for org in orgs])]
orgs.extend(lowercase_orgs['typo'])
typ = typ[~typ['typo'].isin([org.lower() for org in orgs])]
len_1 = typ.shape[0]
diff = len_0 - len_1
print("Organisations removed (lowercase): " + str(diff))
len_0 = len_1

Unique typos: 7997
Organisations removed: 284
Organisations removed (lowercase): 35


In [76]:
capitalised_words = [word for word in typ['typo'] if word.isupper()]
acronyms = ['REO', 'LLC', 'GECRB', 'OLND', 'SYM', 'JC', 'DTI', 'CPVC', 'FCU', 'UC', 'FICO', 'CARDSI', 'HSA', 'HELOC', 'TDI', 'CPG', 'USD', 'FIOS', 'XJ'
            , 'CEFCU', 'PIF', 'HVAC', 'CVS', 'ETS', 'APY', 'BP', 'WR', 'JPM', 'MH', 'DMV', 'SEO', 'HFC', 'RZR', 'CFP', 'EZ', 'YNAB', 'THB', 'DDA', 'BAE'
            , 'PGA', 'KLR', 'NICU', 'FXR', 'ENR', 'EBAY', 'ETF', 'JWE', 'NOY', 'SAIC', 'GMC', 'GPS', 'DS', 'DMP', 'HWCU', 'NCLEX', 'DRW', 'JMH', 'HD', 'GH'
            , 'EIN', 'IVF', 'PMI', 'PITI', 'HOA', 'RTP', 'GSF', 'WFFNB', 'WFNNB', 'TU', 'DIY', 'TGIF', 'TCU', 'ACCTS', 'ELE', 'MSC', 'PV', 'WD', 'RGN'
            , 'LOC', 'GPW', 'AES', 'KJ', 'VNA', 'MFCU', 'SSG', 'SL', 'AWD', 'LX', 'CHP', 'FPU', 'GSXF', 'CSY', 'TRLR', 'GW', 'DM', 'DX'
            , 'AGI', 'CALSTATE', 'FIA', 'MCU', 'VSAC', 'TUCIC', 'DVD', 'DTE', 'ACURA', 'DIL', 'GTI', 'ADME', 'RTQ', 'PMP', 'RGE', 'EER', 'LSU', 'SSN'
            , 'CHGO', 'NHSC', 'AMD', 'SBT', 'FC', 'SSDI', 'ESPP', 'CSA', 'DMAG', 'PMXG', 'CJ', 'SLT', 'MSFCU', 'ZP', 'XB', 'NYS', 'RG', 'XR', 'NCU', 'BSN'
            , 'EE', 'TUC', 'EFX', 'UW', 'NSX', 'ACH', 'OTI', 'WF', 'ZX', 'HDHP', 'RK', 'ALS', 'CVT', 'VTX', 'APRS', 'TUONO', 'DSL', 'GEMB', 'SSI', 'USAFR'
            , 'YTD', 'FX','OKC','DFW','PECI','DEDT', 'SG', 'CRM', 'SDG', 'CCL', 'MRSA', 'SGM', 'BAC', 'POS', 'TMS', 'NYSDOT', 'IRP', 'KYU', 'IRB', 'MSB'
            , 'EO', 'GL', 'BOPP', 'PTSD', 'CWIE', 'FFL', 'TTO', 'NHA', 'ARRA', 'EHR', 'BAM', 'NYU', 'CRC', 'PPG', 'SOA', 'FTASAP', 'FFAE'
            , 'SER', 'RET', 'ARET', 'PIMA', 'JRV', 'CSR', 'CCSD', 'RIF', 'LANTEC', 'JK', 'VSG', 'THP', 'PHR', 'BSEE', 'EMI', 'API', 'DBA'
            , 'ASR', 'SACU', 'APAP', 'UOP', 'GTH', 'UTC', 'EHS', 'MOASC', 'EQR', 'DSG', 'ENAGIC', 'CPT', 'SECU', 'HH', 'AIG', 'NDT', 'FBA', 'SUNUS', 'OPC'
            , 'IPO', 'PRE', 'UNFCU', 'CNOR', 'LLP', 'AVP', 'TCABC', 'SSID', 'QCC', 'JCB','EQ', 'ING', 'SNE', 'FP',  'OPERS', 'RIAZ', 'RFID'
            , 'XLE', 'YHJF', 'SAAS', 'ADT', 'CRX', 'CFNA', 'SSL', 'SSD', 'WCCUSD', 'MSKCC', 'DMS', 'SPX', 'ATT', 'NISCAYAH', 'ASCP', 'TMAT'
            , 'CTS', 'AMR', 'AEP', 'USPTO', 'XLT', 'TT', 'SFPUC', 'FAULTIY', 'CCARD', 'APS', 'ASLO', 'RSU', 'ACK', 'JCP', 'REFI', 'VH', 'SHSAT'
            , 'GRE', 'GMAT', 'NSIT', 'KSL', 'BML', 'HEL', 'GBP', 'GF', 'DAVISON', 'COFS', 'RTG', 'FMV', 'TFS', 'VC', 'DSNB', 'VSP', 'DNB', 'PAYDEX', 'DIT'
            , 'TNB', 'MBM', 'BUIL', 'NGV', 'PTR', 'GSX', 'VPS', 'TXU', 'UPB', 'SOX', 'CCCU', 'MCSE', 'WMC', 'GFI', 'SLC'
            , 'USF', 'MOBILEX', 'BPC', 'MLM', 'SVP', 'PIMI', 'DL', 'CMU', 'TTU', 'BAL', 'PYMT', 'GRSB', 'BD', 'BTH', 'FDNY', 'PAVERS', 'PCI'
            , 'KTM', 'STRS', 'SQL', 'HMC', 'FASFA', 'EMG', 'EDMC', 'PMA', 'DFCU', 'LASIK', 'BYU', 'APPT', 'UNM', 'BECU', 'CFA', 'PMT'
            , 'UAB', 'STUC', 'ISC', 'ISPC','EIP', 'FAKO', 'PPA', 'WPPI', 'CTM',  'FSA', 'OCSD', 'RO', 'EAM', 'RBM', 'FSU', 'YZF', 'ERP', 'CFE', 'RHS', 'MPA'
            , 'PGE', 'FSH', 'TMJ', 'CONUS', 'AWI', 'VIT', 'PTZ', 'XLG', 'HDR', 'SRT', 'MRO', 'FIU', 'DECU', 'VZ', 'KCPD', 'BAYADA', 'RFP', 'DHS', 'FML','SRE'
            , 'WSM', 'GYN', 'BELK', 'BFC', 'EG', 'FDOT', "'KMA",'NYSTRS', 'FLETC', 'XT', 'EMR', 'AGN', 'HELCO', 'MSOE', 'WTH', 'PLTW', 'IEA', 'TEP'
            , 'OSU', 'KPF', 'HOK', 'LUMI', 'MSN', 'CRF', 'NOI', 'AE', 'USAR', 'KLOC', 'FSFSA', 'MRP', 'SDVOSB', 'CP', 'GSXR', 'ISP', 'ATL', 'VMI', 'COCKRELL', 'ARHCU'
            , 'GNG', 'TALCE', 'CRE', 'IAAVS', 'PSU', 'DCR', 'GBS', 'EJ', 'GTO', 'NCT', 'PBW', 'MLB', 'GMAIL', 'MLS', 'ARMAC', 'JNKCV', 'WVO', 'CNY'
            , 'USPCC', 'RLU', 'WIFI', 'CUNY', 'WH', 'OBX', 'POC', 'BOM', 'JVC', 'GY', 'SES', 'EDU', 'DRI', 'RSS', 'RTO', 'BHPH', 'JIC', 'QE', 'DDR'
            , 'QSOL', 'APA', 'BT', "YE'", 'REI', 'SOFIJA', 'DSW', 'MCSA', 'ISU', 'CCI', 'ADR', 'PDF','CSU', 'DSLR', 'IP', 'VOC', 'RIT', 'LLM', 'AIA', 'PNW', 'BH'
            , 'APK', 'GUSD', 'GFE', 'BNY', 'LSAT', 'LOA', 'IMS', 'DFS', 'PRM', 'ACS', 'GRT', 'PUD', 'CCS', 'NRG', 'JCL', 'QMRP'
            , 'CCNA', 'DVT', 'BCC', 'JBL', 'ZD', 'ITN', 'AMX', 'USACII', 'USACCI', "FY'", 'CQF', 'SJSU', 'LVN', 'TMJD', 'LV', 'OPB', 'DTL', 'ESPINDOLA', 'UVA', 'SEP'
            , 'SEM', 'PULISH', 'CSET', 'CBEST', 'PLS', 'GMAX', 'DFACS', 'Oll', 'APR', 'SUV', 'USAF', 'WWW', 'ATV', 'NYC', 'SCCU', 'NASDAQ', 'HTTPS', 'VRHS'
            , 'UFCS', 'BMW', 'CPA', 'ISA', 'IRS', 'IRA', 'ATM', 'EMT', 'SSA', 'FHA', 'RSVP', 'OSHA', 'NACA', 'SAM', 'USA', 'MSA', 'MRI']
typ = typ[~typ['typo'].isin(acronyms)]

len_1 = typ.shape[0]
diff = len_0 - len_1
print("Acronyms removed: " + str(diff))
len_0 = len_1

Acronyms removed: 516


In [77]:
# Look at words starting with capital letter and remove non-typos
capitalised_words = [word for word in typ['typo'] if word[0].isupper()]

entities = ['Citi', 'LendingClub', 'Transunion', 'Experian', 'USbank', 'DiscoverCard', 'APRs', 'Bal', "CC's", 'LENDINGCLUB', 'STARTUPSNAX', 'NYCTRINITY', 'CBGONZALEZ' 
            , 'USBank', 'Baloon', 'Verinik', 'WARESALE', 'ZENSAH', 'NUSBAND', 'TRANSUNION', 'SUGARSAND', 'RATON', 'DADELAND', 'SHOPSTORES','WELLCRAFT', 'INGROUND', 'SURBURBAN'
            , 'Eloan', "Raley's", 'Broward', 'AmEx', 'Zoso', 'MedAssets', 'Lowes', 'Jetta', 'BoA', 'Pennys', 'CREDITALS', 'Sportsmans', 'TransUnion', 'MOTORHOME'
            , 'Happypurpose', 'Happycredit', 'Lasalle', 'Durango', 'Dodd', 'Amex', 'Springleaf', "Francisco'", 'BlueBook', 'Sharebuilder', 'CROSSFIT', 'USBANK', 'NEHOTDOG'
            , 'Toysrus', "Reserve's", 'Samsung', 'Poolboss', 'Motorhome', 'Pepperdine', 'Refi', 'Hepa', 'LinkedIn', 'YouTube', 'GooglePlus'
            , 'NexTier', 'Zillow', 'CreditCards', 'CCs', "APR's", 'BofA', 'Px', 'Lancruiser', 'CreHigh', 'Wa', 'Suze', 'Orman', 'Cabelas', 'Hodgkins'
            , 'Macys', 'Penneys', 'Sams', 'Crownline',  'Holsteiner', 'Fauxwood', 'Telecom', 'CapOne', 'Nordstroms', 'Aprilia', 'Shovelhead', 'Soffits', 'AAdvantage'
            , 'Supercrew', 'Levantina', 'MacBook', 'Yaris', 'Hidenwood', 'Fico', 'Ipay', 'Prostodontics', 'Endodontics', 'Pune', 'OneMain'
            , 'Brookwood', 'Paydown', 'QuickBooks', 'ProAdvisor', 'SilverDollar', 'Airboat', 'Derebery', "Jones's", 'HomeDepot', 'Kawa', 'Ponte', 'Vedra'
            ,  'Rosso', 'Elec', 'Virco', 'Moly', 'Ues', 'CCards', 'Sportster', 'Citicards', 'Citflex', 'Kenai', "HOA's", 'Ironmans', 'Cranbrook'
            , 'Citicard', 'GeMoney', 'Truecredit', 'Iplan', 'Uhaul', 'MacbookPro', 'AdWords', 'AdCenter', 'Chartplotter', 'Consolidatin', 'Inground', 'Davidsons'
            , "DVD's", 'NewEgg', 'Newegg', 'Depts', 'Acetylyne', 'Drupal', 'Lendingclub', 'Mgmt', "SKU's", 'NGOs', 'Eller', 'Waverunners', 'Freightliner', 'DeVille'
            , 'Cyl', 'Photovoltaic', 'Enphase', 'Kickstarter', 'ConEdison', 'Ebay', 'Rolex', 'CapitalOne', 'BestBuy', 'UHaul', 'NMCCs', 'MBAs', 'Util', 'Biotech'
            , 'Pacifica', 'Capitalone', 'Eqifax', 'Gooseneck', 'CitiCard', 'GoogleTV', 'Denali', 'Sixeas', 'Morro', 'Obispo', 'Mics', 'Lakeview', 'Silverado', 'Ducati'
            , 'Burea', 'DoD', 'Whitworth', 'Howards', 'ATMs', 'Wi', 'Doughboy', 'Meetup', 'Groupon', 'App', "ON'S", 'Lufkin', 'TJMaxx', 'Stomas', 'Tubo'
            , 'Injecter', 'Reatta', 'Ater', 'Trailmanor', 'VINEYL', "HOUSE'", 'AmexCard', "EV's", 'Halliburton', 'MegaCab', 'Cummins', 'Sienne', 'Speedmaster'
            , 'Semper', 'Fi',  'Starcraft', 'Vespa', 'Licence', 'Citicrad', 'SallieMae', 'Aquarian', 'Carecredit', 'Barclays', 'Corrolla', 'Countertops', 'Rv'
            , 'MyFICO', 'FLa', 'Altima', 'WiFi', 'Opry', 'Promissary', 'Maint', 'Camaro', 'AmeriCorps', "LendingClub's", 'Wal', 'SolarCurrents', 'Miscel'
            , 'Online', 'Discovercard', 'Prius', 'Tks', 'Fairwinds', 'Bowflex', 'Roamans', 'Deby', 'CareCredit', 'Juris', 'Experion', 'Boomers', 'SolarCity', 'Soboba'
            , 'Neiman', 'Acura', 'Bally', 'Pilates', 'Joes', 'Bly', 'Gunite', 'Pennsauken', 'Adwords', 'Exquifax', 'Worldmark', 'Chillicothe', 'Starscapes', 'Hardie'
            , 'Craigslist', 'Fresenius', 'Spooner', 'Sundancer', 'Phuong', 'Sturgis', 'Kawsaki', 'Ludlum', 'Tha', 'ServiceLink', 'Zumba', 'Motorsports', 'Oneonta', 'Thany'
            , 'BankAmericard', 'Permananete',  'Homedepot', 'Hava', 'Virto', 'Lookings', 'Jamall', 'Elmendorf', 'Nitro', 'Brammo', 'Citiibank', 'Skidoo', 'Terminix'
            , 'Volusia', 'Pharma', 'Skidsteer', 'RealPage', 'Obama', 'Tuono', 'DVDs', 'Lunds', "Byerly's", 'Goldwing', 'Bic', 'Flippa', 'Pipeliner', 'Clight', 'Celica'
            , 'WinePressGroup', 'Neuroscience', 'Bridgpoint', 'Reattas', 'Wellsfargo', 'SpeechEasy', 'Djs', 'Jaytee', 'Arabica', 'Landstar', 'Aslo', 'Medway'
            , 'Rucker', 'Tictok', 'Cuyahoga', 'MRIs', 'Eldy', 'Fatboy', 'SteakNShake', 'Lumina', 'Razorfish', "Chron's", 'CPAs', 'Pharm', 'CrossFit', 'Stroudsburg', 'LendingCLub', 'Kretek'
            , 'Arblebeedie', 'Beelte', 'Paintball', 'Blogs', 'Youtube', 'Flickr', 'QuinnipiacUniversity', 'Chula', 'Saks', 'Deltacom', 'NuVision', 'Springville'
            , 'Clackacraft', 'Vuitton', 'Moet', 'Hennessy', 'Husqvarna', 'Cleanpro', 'Westfalia', 'Volkswagon', 'Salesforce', 'CIOs', 'Uinta', 'Menards', 'Daytona'
            , 'Chennai', 'Changs', 'Rhis', 'Cummington', 'Ramapo', 'Rhinebeck', 'Laguardia', 'Biodot', 'Kiva', 'Minumum', 'Santander', 'Maxima', 'GMoney', 'Gmoney'
            , 'Lukes', 'Wadell', 'Duathlon', 'DasM', 'Sno', 'Merita', 'WorldMark', 'Pre', 'Phd', 'JETBOAT', 'KonicaMinolta', 'Juilliard', 'Greatwide'
            , 'ClearPointe', "Crohn'", 'Greiner', 'Duckman', 'ETFs', 'Ishares', 'Patrisia', 'GEMoney', 'JPMorgan', 'Dicover', 'Opgal', 'Villanova', 'Wamu'
            , 'Airstream', 'WellsFargo', 'Fiserv', 'AlwaysOn', 'Vallarta', 'Alcapulco', 'Esthetician', 'Un', 'Exoress', 'PeaceHealth', 'Apnea', 'Haistar', 'Festivus'
            , 'CreditKarma', 'Aldi', 'Buckhead', 'Regence', 'PocketSmith', 'Aliant', 'Monitronics', 'Cheernastics', 'Jarden', 'Moto', 'Az', 'ThomsonReuters'
            , 'BelAir', 'Kinesiology', 'Neorstar', 'SearsCreditScore', 'Ameriprise', 'CashReturns', 'Demusz', 'Fios', 'Helzberg', 'Lemans', 'Darden', 'Novasure'
            , 'Sephora', 'EurAuPair', 'Agi', 'Dvd', 'Dupage', 'Phils', 'LCs', 'HearUSA', 'Lasik', 'Hanford', 'Redbox', 'DVDNow', 'Nori', 'Mattingly', 'Nutritionals'
            , 'Evercleanse', 'Enagic', 'Pascagoula', 'AmWINS', 'Rostami', 'LClub', 'Francetta', 'Loius', 'Branson', 'Mauldin', 'Armedia', 'Ikea', 'Tallahasse'
            , 'Chickfila', 'Aarons', 'Soalr', 'Formostar', 'Ameritrade', 'Frontsight', 'CRv', 'Hempfield', 'Acromegaly', 'Cephalon', 'Blog', 'FriendFinder'
            , 'Markarian', 'Wedgewood', 'Pandit', 'Keuka','Sping', 'Pers', 'Exterran', 'Hbsc', 'AutoCAD', 'CreditSecure', 'Lanaii', 'PRs', 'Haywayrd'
            , 'Vianina', 'Raymour','Flanigan', 'Pricegrabber', 'Dorathy', 'Amblin', 'Lae', 'Gavilon', 'Rewardzone', 'Quattro', 'WordPress', "Haverty's", 'BjB', 'Cas'
            , 'Paratransit', 'Kompressor', "Oletello's", 'Infiniti', 'MVelopes', 'Accenture', 'Puliz', 'Hibdon', 'Walla', 'Pannels', 'Councelor', 'Ridgeline'
            , 'Loganville', 'Genco', 'Petco', 'Mifflin', 'SharePoint', 'Waverunner', 'Waverrunner', 'Xcentric', 'LexisNexis', 'Cryptologics', 'Permanente'
            , 'Taubman', 'Fairlane', 'Spokanes', 'Raton', 'Quizzle', 'Phenom', 'Electraglide', 'Kahren', 'Bentonville', 'FirstBank', 'LLBean', 'Weisfield'
            , 'Flitestar', "Jones'", 'Crd', 'Opici', 'Aetna', 'CaptialOne', 'Hazmat', 'Chevelle', "Sear's", 'JcPenny', 'VIsa', 'Bestbuy', 'Xbox', 'Iphone', 'Symbian'
            , 'Bens','Mitsui', "Kinko's", 'TrueCredit','Alpharetta', 'MasterCards', 'Fingerhut', 'Edwardsville', 'Coldwell', 'Realogy', 'Greewood', 'Bayliner'
            , 'Softtail', 'PreK', 'Providian','Ultrasonographer', 'Nextel', 'Usbank', 'Charlevoix', 'Petoskey', 'Mamaroneck', 'Jacksboro', 'Revoling', "LC's", 'HELOCs'
            , 'Nagahama','Captil', 'Landline', 'NetJets', 'Birkshire', 'Converseo', 'Citifincial', 'Tarrant', 'REOs', 'Palmatte', 'Riteway','Yoon', 'Bon', 'CitCard','GIBill'
            , 'Willys', 'Bikram', 'Marland','Corvair','Ratha','CareOne', 'Velux', 'Horicon', 'DIsh', 'Technition', 'Elliman', 'Wilm','Experien', 'Lotawana', 'Azusa'
            , 'GeoThermal', 'GoeThermall', 'Tencor', 'Jayco', 'Wairarapa', 'Sauv','Bellevue', 'Harborview', 'Groveport', 'Doula','Arbonne', 'Impreza', 'Fonality', 'Xpres'
            , 'Repulic', 'Hidra', 'Ckass', 'Phoenis', 'Dadeland', 'CriticalMiss', "Fiancee'", 'ExperianCredit', 'CNNMoney', 'ProSci', 'AntiCancer', "Sleepys'", 'MediMedia'
            , 'Insur', "Northwestern's", 'Tyra', 'CCConcepts',  'Tec', 'Debutante', 'Tradewinds', 'Ipad', 'CDs', 'BerkleeMusic', 'Tourneau', 'Wellfargo', 'Sampco'
            , 'EnerBankUSA', 'Shoprite', 'Deloitte', 'Targa', 'Montefiore', 'Taylorville', 'Bof', 'Oryon', 'Manatt', "O'Melveny", 'Vumii', "PO'd", 'Barclarys', 'Tradeshow'
            , 'Brighthouse', 'Macbook', 'MacDill', 'Engelbreit', 'InBev', 'SuperBowl', 'Yakira', 'Lemoore', 'Danise', 'Corix', "VP's", 'McArdle', 'Integra', 'Brea'
            , 'Celink', 'CreditClub', "CD's", 'DJs', 'Plex', 'Cama', 'Appx', 'Granbury', 'Echocardiography', 'Sqft',  'Crestron', 'Grooveshark', 'UMass', 'Stonehill'
            , 'Shinwa', 'Victorias', 'Lovetri', 'Temecula', 'Bluewater', 'Watauga', 'Massons',  'Skydrol', 'Lan', 'Panera', 'Wipfli', 'Keiser', 'FullSail'
            , 'UChicago', 'McNair', "Starbuck's", "Children'sHospital", 'Automall', 'Pinellas', 'VisaCard', 'Teksystems', 'Telerx', 'Eldercare','Hujiang', 'Falmouth'
            , 'Dj', 'Roto', 'Tula', 'Viscira', 'VitesseLearning', 'Tv', 'SoHo', 'Cobham', 'DelMarVa', 'Oth',  'Sugarland', 'AADvantage', 'Terracon', "CEO's", 'Botteling'
            , 'Suisse', 'CalPERS', 'Transcriptionist', 'Bloomingdales', 'Forex', 'Elan', 'Petropolis', 'Voc', 'Internt', 'SUVs', 'FairLenderUSA', 'Waikele'
            , 'CSRs', "Obama's", 'Precor', "Cummuta's", 'Sccfcu','Aidance', 'CLub', 'INGDirect', 'PwC', 'GapCard', 'Abercrombie', 'Solopreneurs', 'Nambia', 'Voip'
            , 'CitiFinance', 'Anderso', 'Biomed', 'Ewings', 'Oxymorons', "Batter's", 'McKinsey', "MRI's", 'Reddit', 'DeVry', 'Belvoir', 'Eldor', 'Euro'
            , 'Delray', 'AmarOil', 'Welch', 'CableOne', 'Alagasco', 'Agrifos', 'WalletPop', 'Unifed', 'Macdill', 'Osan', 'TheraCom', 'Bofa','QuadKing', 'Arthurs'
            , 'Awfularthursseafood', 'Kosovo', 'TruGreen', 'Quozzy', 'Grande', 'Metalcraft', 'Eisai', 'Booz', 'Kenexa', "SRE's", 'Conica', 'Alos', 'Zeor'
            , 'Timberlodge', 'Flannigan', 'Utilites', 'DMGetDocument', 'LexisNexs','GetFugu', 'Ashford', 'Sealift', 'LaJolla', 'Sause', 'Rehoboth', 'TreeFarmer', 'Mactec'
            , 'HighSpans', 'Maricopa', 'Ymca', 'Pricewaterhousecoopers', 'Femoroacetabular', 'Rubilotta', 'Copey', 'Desku', 'Rishi', 'Bhat', 'Telsmith', 'CommID'
            , 'Yahama', 'Eagan', 'Acitve', "Thacaro's", 'Becon', 'Sedona', 'Poolcraft', 'Gensler', 'Poly', 'Kyomatt', 'Moviebox', "Moviebox's", 'Blu', 'StocksMan'
            , 'HealthCentral', 'CashFlow', 'Ridgewood', 'Alaskian', 'Malmute', 'Inder', 'Prodigo', 'Ramsy', 'Gallaudet', 'OnSite', 'Integreon'
            ,  'Tulum', 'Towncar', 'Blackhawks', 'Edelbrock', 'Amanti', 'Tx', 'Crohns', 'Taekwondo', 'Ridgeback', 'Learjet', 'Barryville', 'Praveen', 'BigDog', 'Menendez'
            , 'Kamran', "Momma's", 'Somatics', 'VanKeuren', 'Broco', "Rubio's", 'Ucla', "Rhode's", 'Napa', 'Deskin', 'Metroplex', 'Selem', 'Sholin', 'Dass'
            , 'Sonographer', 'Pecnik', 'Marymount', 'Ghazi', 'Swann', 'DeBrino', 'MorphBrand', 'Anahiem', 'Gerig', 'Yaworsky', "IR's", 'Rhode', "Bug'", 'Medaxial'
            , 'DebtFreeAdventure', 'Ramen', 'Havastat', 'Aquiring', 'Loc', 'Chia', 'Lerma', 'Deepwater', 'Geschwindt', 'Renior', 'Bellingham', 'DingKing', 'Imtiaz'
            , 'Fernie', 'Goico', "RV's", 'Schlenz','Bio', 'Khecari', 'Newsteps', 'Flieger', 'GoGoVertigoat', 'BoCoCa', 'Mancuso', 'Natchitoches', 'FritoLay', 'Anc'
            , 'Canisius', 'Curesome', 'Kehoe', "RN's", 'Akerstein', 'Regolith', "Aug'", 'Niguel', 'Girbes', 'Markesia', 'Akinbami', 'Sampsel', 'Mercruiser'
            , 'Choate', 'Paseo', 'Brzostowski', 'Centeno', 'Regenexx', 'Kamdolla', 'Collums', 'Kenwood', 'Asano', 'Taiko', 'Hono', 'Hanes',"Alphie's", 'NextGen', 'Alaaddin'
            , 'Braikbrothers', 'Arborist', 'Sonography', 'Turnage', 'Garbow', 'Umpqua', 'Hatchett', 'Tysabri', 'Mudd', 'Disneycruise', 'Janovsky', 'Ponitac', 'Lendin'
            , 'Goldbar', 'Obert', 'Edmunds', 'PowerShares', 'WilderHill', 'Jc','Materail', 'Bramwell', 'Dmitriy', 'Phy', 'Asplundh', 'TULESFROZENYOGURT', 'Glavin', 'Rosner'
            , 'Lewallen', 'SSgt', 'Whe', 'Adhish', 'Paydex', 'Daihatsu', 'Mastergeorge', 'MacMurray', 'Tiffaney', 'LeadAmerica', 'ARMs', 'DollarServ'
            , 'Valento', 'Bair', 'CreditEducation', 'WhatsInYourScore', 'Jumpshot', 'Tribeca', 'Merceds', 'Kona', 'Saracino', 'Twan', 'RogerGrady', 'LLCs'
            , 'PricewaterhouseCoopers', 'Citimortgage','CDOs', 'ARMSs', 'Zurich', 'Hayabusa', 'Hsbc', 'Citifancial', 'Hodder', 'Dezendorf', 'Enfield', 'Franske', 'Petrie'
            , 'Baij', 'McQueary','Viccaro', 'Nav', 'Kenner', 'LWLMedia', 'Banknorth', 'BioDiesel', 'Biodiesel', 'Linconl', 'SKUs', 'JEPook', 'Xb', 'Deville'
            , 'VocRehab', 'Trooptastic', 'Appleman', 'Belfair','Grapeview', 'Kitsap', 'Bremerton', 'Oviedo', 'WebSuccessAgency', 'SaaS', 'SaaC'
            , 'SalesForce', 'Marleny','Groza', 'RDMc', 'RDMcPublishing', 'Archicad', "GMAT's", 'Kubota', 'Surger', 'Locums', 'Lesabre', 'VisitorsGuide', 'NorthBeachSun'
            , 'Lowrey', 'Petworth', 'Slattery', 'Tingen', 'Jivox','Anthropolgy', 'Blas', 'Levitz', 'NetworkPlusConsulting', 'BusinessPlanforGSITF', 'Binu'
            , 'Eapen', 'Aviastroitel', 'Weizen', 'Shapewear', 'Bodyshaping', "HUSH's", 'Murtza', 'Naseem', 'Xterra', 'GMATs', 'Humana', 'BlueStarTees'
            , 'FolioFN', 'AffiliateDiamond', 'TutoringOne', 'WebChat', 'Topolski', 'Marist', 'Bac', 'Furthersite', 'Secretarys', "SUV's", 'Wintersville'
            , 'Alibaba', 'Powerseller', 'VoIP', 'StartupSnax', 'Munchables', 'Ebook', 'Ebooks', 'PublishersWeekly', 'Hazlewood', 'Sanitaire', 'Eastover', 'Maxfield'
            , 'Nirav', 'Subrau', 'Tamie', 'Opteron', 'Psy', 'Carrollton', 'Jasniewski', 'Mcbee', 'Kripalu', 'Vero', 'Soloman', 'Mazdaspeed', 'Pallisades', 'Rehana'
            , 'Husain', "Orman's", 'Stott', 'Bluenile', 'Nordstorm', 'Charnel', 'Everly', 'Newberry', 'Credet', "SIL's", 'DandD', 'DandDToys', "ATV's", 'VinyasaMT'
            , 'Barack', 'Sawgrass', 'Banos', 'SoCal', 'Touche', 'Sempra', "MBA's", 'Lithia', 'Wai','Ipods', 'Nano', 'Shenzen', 'Toro', 'VISAs', 'Ser', 'Cheareen', 'INTj'
            , 'Linktank', 'Brentwood', "Reports'", 'HappyBalls', 'Kalata', 'Berend', 'Becu', 'Institue', 'Ecoli', 'Cabe', 'Microfinance', 'Hilal', 'Homaidan'
            , 'CollegeClassifieds', 'Zensah', 'Enloe', 'BillMeLater', 'Kangan', 'Dti', 'Nj', 'Lipoma', 'Golfsmith','Atronic', 'Escows',  'Ent', 'Drs', 'Northstrom'
            , 'Zales', 'CardsCapital', 'Blueline', 'Longaberger', 'Xceed', 'Wiki', 'Issaquah', 'AtriCure', 'Coolrail', 'Lumitip', 'Frigitronics', 'Cryo', 'Patt'
            , "Amex's", 'CreditSecur', 'Emich', 'Acc', 'Amaroil', "Fico's", 'Biopharmaceutical', 'Bandera','Lindenwood', 'Sportage', 'Groc', 'Jus', 'Biomolecular'
            , 'Bain', 'Tigran', 'Danielyan', 'Amberton', 'Statoliner', 'Ericson', 'Shillington', 'Papercutz', 'Bionicle', 'Reatin', 'Mouw', 'Abetterwatobuy'
            , 'Abetterwaytobuy', 'Nikoli', "Nikoli's", 'Vo', 'Nieto', 'Tama', "EMT's", 'Fosgate', 'Accoustic', 'Morgantown', 'Clarksburg', 'Neo', 'QQitemZ'
            , 'QQcmdZViewItemQQptZUS', 'Diaz', 'Collazo', 'Nicolau', 'Fortlauderdale', 'MesmoTV', 'ImproveTrafficNow', 'Primus', 'Jani', 'Legue', 'Flagg', 'Chaitanya'
            , 'GoZone', 'Baltazar', 'Tencel', 'Bulkin', 'Temporomandibular', 'Townhome', 'Boynton', 'Fidelis', 'Mpls', 'DOWNOUR', 'Ibanez', 'McAllen', 'Kirksville', 'Relo'
            , 'Benning', 'DQs', 'Msc', 'Hawkeye', 'Melrose', 'Hille', 'Bora', 'ServiceMagic', 'AdrenalinIsland', 'DjScottyd', 'Rebecca', 'Hassett', 'ThePlanet', 'Beltone'
            , 'Ferotec', 'Raybestos', 'Manheim', 'Zayin', "DQ's", 'Spina', 'Bifida', 'Broadmoor', 'Equafax','Frid', 'AuroraLove', 'AnnaCFarney', 'Farney'
            , 'ViewUserPage', 'Hillsborough', 'Graco', 'RogerFarney', 'Hotmail', 'Sentra', 'Euros','Darien', 'Pell', 'Cuny', 'Portuguez', 'McCombs', 'Komodo', 'Internet'
            , 'Geico', 'Citifinancial', 'Netflix', 'Miata', 'Citigroup', 'Walmart']
typ = typ[~typ['typo'].isin(entities)]

#Add in those detected in Stanford NER
entities.extend(orgs)

In [78]:
e = typ.loc[typ['typo'].isin([entity.lower() for entity in entities]), 'typo']
a = typ.loc[typ['typo'].isin([acronym.lower() for acronym in acronyms]), 'typo']

typ = typ[~typ['typo'].isin([entity.lower() for entity in entities])]
typ = typ[~typ['typo'].isin([acronym.lower() for acronym in acronyms])]

entities.extend(e)
acronyms.extend(a)

In [79]:
# Payer is a word 
typ = typ[~typ['typo'].isin(['payor','payer', 'Payor', 'Payer'])]
# Most US keyboards don't allow for accents, so ignore typo corrections with accents, e.g. fiancee and clientele
typ = typ[typ['recommended'].str.contains('è') == False]
typ = typ[typ['recommended'].str.contains('é') == False]
typ = typ[~typ['typo'].isin(['Cafe', 'cafe', 'fiance', 'fiancee'])]
# Words starting with or ending with ' were falsely listed as typo
typ = typ[typ['typo'].str.contains("'") == False]

len_1 = typ.shape[0]
diff = len_0 - len_1
print("Non-typos removed: " + str(diff))
len_0 = len_1

Non-typos removed: 1795


In [80]:
slangs = ['Yappy', 'Btw', 'Doggie', 'LOL', 'lol', 'PLS', 'WooHoo', 'Hmm', 'Aprox', 'Thx', 'Yay', 'Thanx', 'Woohoo', 'Flippin',  'HOOAH', 'WOOOOOO', 'OMG'
          , 'SOOOO', 'Feh', 'WAAAY', 'Whoohoo',  'Lol', 'Wowww', 'Yessss', 'Goooo', 'Plz',  'Argh',  'XXXXX', 'XXXXXX',  'PLEEASSSEEE', 'Dawgs','Ahhhh']

shorthands = ['Desc', 'Pmt',"Gov't", 'Pmts', 'Xwife', "Int'l", 'Intl', 'Pymt', 'Admin', 'admin', 'Pymts', 'pcm', 'FYI', 'bdrm']

medical_terms = ['Osteomyelitis']

typ = typ[~typ['typo'].isin(slangs)]
typ = typ[~typ['typo'].isin(shorthands)]
# Add lowercases
slangs.extend(typ[typ['typo'].isin([word.lower() for word in slangs])]['typo'])
shorthands.extend(typ[typ['typo'].isin([word.lower() for word in shorthands])]['typo'])
typ = typ[~typ['typo'].isin([word.lower() for word in slangs])]
typ = typ[~typ['typo'].isin([word.lower() for word in shorthands])]

In [81]:
import warnings
warnings.filterwarnings("ignore")

# Urban dictionary API does not work - HTTPError 429 (too many requests)
# Source: https://github.com/bocong/urbandictionary-py/blob/master/urbandictionary.py
# Fixed a bug (https://github.com/bocong/urbandictionary-py/issues/8)
# Load manual csv file, found in Kaggle
# Note: some definitions have commas and throw errors, but the full definition isn't needed so ignoring warning

urbandict = pd.read_csv("urbandict-word-def.csv", sep = ",", error_bad_lines=False, warn_bad_lines=False)
urbandict.head()

,word_id,word,up_votes,down_votes,author,definition
0,9.0,yayeeyay,19,27.0,dc397b2f,"affirmation; suggestion of encouragement, appr..."
1,7.0,Janky,296,255.0,dc397b2f,Undesirable; less-than optimum.
2,13.0,brutal,12,45.0,40ece1ef,anything that makes you sweat
3,14.0,skanky,9,48.0,485e4db7,"Anything of or pertaining to a $10,000 hooker."
4,22.0,hecka,8,18.0,b9dcf126,see synonyms at hella.


In [82]:
urbandict.loc[urbandict['word'] == 'dissapointed'] 
#Urban dictionary has a variety of common typos


,word_id,word,up_votes,down_votes,author,definition
1469027,"4,064,666.0",dissapointed,89.0,17.0,31c5f6a3,The wrong way to spell disappointed.


In [83]:
urbandict.loc[urbandict['word'] == 'ect'] 
#Some words have multiple definitions, one of which is misspelling

,word_id,word,up_votes,down_votes,author,definition
228618,"562,818.0",ect,104.0,50.0,ca8d8385,electroconvulsive therapy. an electric current...
386595,"1,029,419.0",ect,332.0,151.0,19b2b06,speaking of stupid people... its actually spel...
507401,"1,332,145.0",ect,114,49,1d65c1cd,bastardisation of 'etc.' or et cetera.;;;;only...
1698277,"4,798,028.0",ect,5,51,ba9b2a72,ect is often made to look like a person with a...
2582182,"8,642,277.0",ect,1,0,262e4f73,ect. is an often falsely used abbreviation for...


In [84]:
urbandict.loc[urbandict['word'] == 'youre'] 
# The ones with few upvotes are likely to be typos

,word_id,word,up_votes,down_votes,author,definition
26794,"65,778.0",youre,8,34.0,431d0462,New Zealand spelling of the word 'you're'.
86221,"197,651.0",youre,11,75.0,3b24ff6d,your
282345,"741,050.0",youre,71.0,10.0,89bc4d7b,1) used interchangeably to mean either 'your' ...


In [85]:
urbandict['up_votes'] = pd.to_numeric(urbandict['up_votes'],errors='coerce')

In [86]:
urbandict = urbandict.loc[urbandict['up_votes'] > 20] # Selected 20 - out of scope but would be interesting to see what the cut-off is for slang vs. error

In [87]:
urbantypos = set(urbandict.loc[urbandict["definition"].str.contains("spell|typo|illiterate", na = False), 'word'])

In [88]:
urbanwords = set(urbandict['word'])

In [89]:
urbanwords = urbanwords - urbantypos

In [90]:
urban_typo = typ[typ['typo'].isin(urbanwords)]
urban_typo = urban_typo[urban_typo['typo'].str.lower() != urban_typo['recommended'].str.lower()] # remove the ones where the only difference is the capitalisation, e.g. christmas, june
urban_typo.head()

,typo,recommended
39,nbsp,tbsp
84,nevermind,never mind
119,fo,few
131,brainer,briner
290,Hoorah,Howrah


In [91]:
len(typ.loc[typ['typo'].isin(urbanwords), 'typo'])
# Add these to slang
slangs.extend(typ.loc[typ['typo'].isin(urbanwords), 'typo'])
len_0 = typ.shape[0]
typ = typ[~typ['typo'].isin(urbanwords)]
len_1 = typ.shape[0]
diff = len_0 - len_1
print("Slangs removed: " + str(diff))
len_0 = len_1

Slangs removed: 586


In [92]:
# repeated letters for emphasis
emphasis = []

from tqdm import tqdm

patterns = [r"so+$", r"wow+$", r"wa+y$", r"ye+s+$", r"ple+a+s+e+$", r"you+$", r"tha+nk+s*$"]

for i, typo in enumerate(tqdm(typ['typo'])):
    for p in patterns:
        pattern = re.compile(p)
        if pattern.match(typo.lower()):
            emphasis.append(typo)

emphasis

typ = typ[~typ['typo'].isin(emphasis)]
slangs.extend(emphasis)

100%|██████████| 4733/4733 [00:00<00:00, 94555.94it/s]


In [93]:
# Some manual corrections of suggestions
typ.loc[typ['typo'].isin(['im', 'Im', "i'm", 'IM']), "recommended"] = "I'm"
typ.loc[typ['typo'].isin(['eg', 'Eg', 'EG']), "recommended"] = "e.g."
typ.loc[typ['typo'].isin(['takingmeplaces']), "recommended"] = "taking me places"

print("Number of unique typos: " + str(typ.shape[0]))

Number of unique typos: 4723


In [94]:
# Get the changes
import difflib
drop_list = []
add_list = []
n_drop = []
n_add = []

for i, row in typ.iterrows():  
    typo = row['typo']
    rec = row['recommended']
    #print('{} => {}'.format(typo, rec))  
    drop = []
    add = []
    if pd.isnull(rec):
        add.append("")
        drop.append("")
    else:
        for i,s in enumerate(difflib.ndiff(typo, rec)):
            #print(s)
            if s[0]==' ': continue
            elif s[0]=='-':
                drop.append(s[-1])
            #    print(u'Delete "{}" from position {}'.format(s[-1],i))
            elif s[0]=='+':
                add.append(s[-1])
            #    print(u'Add "{}" to position {}'.format(s[-1],i))    
    drop_list.append(drop)
    n_drop.append(len(drop))
    add_list.append(add)
    n_add.append(len(add))

typ['drop'] = drop_list
typ['add'] = add_list
typ['n_drop'] = n_drop
typ['n_add'] = n_add

typ['n_changes'] = typ['n_drop'] + typ['n_add']
typ = typ.sort_values(by=['n_changes'], ascending = False)
typ.head(5)

,typo,recommended,drop,add,n_drop,n_add,n_changes
13670,siouxfallsamericandream,Americanization,"[s, i, o, u, x, f, a, l, l, s, a, d, r, e, m]","[A, i, z, t, i, o, n]",15,7,22
18450,bankofamerica,Americanization,"[b, a, n, k, o, f, a]","[A, n, i, z, a, t, i, o, n]",7,9,16
15531,musculoskeletal,musculature,"[o, s, k, l, e, t, a, l]","[a, t, u, r]",8,4,12
16997,thephotobuddy,telephotography,"[h, b, u, d, d]","[e, l, g, r, a, p, h]",5,7,12
243,Justfininshed,Justification,"[i, n, s, h, e, d]","[i, c, a, t, i, o]",6,6,12


### Here, it was output to CSV, and I manually corrected any wrong typo correction recommendations

In [95]:
typF = pd.read_csv("typos_changes_manual.csv", encoding = "ISO-8859-1")
typF.head()

,typo,recommended,medical,not_typo,slang,entity,acronym,shorthand,html,fixed
0,siouxfallsamericandream,Americanization,nan,nan,nan,nan,nan,nan,nan,Sioux Fallls American Dream
1,bankofamerica,Americanization,nan,nan,nan,nan,nan,nan,nan,bank of America
2,musculoskeletal,musculature,nan,nan,nan,nan,nan,nan,nan,musculoskeletal
3,thephotobuddy,telephotography,nan,nan,nan,nan,nan,nan,nan,the photo buddy
4,Justfininshed,Justification,1.0,1.0,nan,nan,nan,nan,nan,just finished


In [96]:
medical = typF.loc[typF['medical'] == 1, 'typo']
medical[:5]
medical_terms.extend(medical)
print("Number of medical terms: " + str(len(medical_terms)))

Number of medical terms: 64


In [97]:
# SLANGS
s = typF.loc[typF['slang'] == 1, 'typo']
slangs.extend(s)
print("Number of new slangs: " + str(len(s)))
print("Number of total slangs: " + str(len(slangs)))

# ENTITIES
len_0 = typF.shape[0]
entities.extend(typF.loc[typF['entity'] == 1, 'typo'])
typF = typF[typF['entity'] != 1]
len_1 = typF.shape[0]
diff = len_0 - len_1
print("Entities removed: " + str(diff))
len_0 = len_1

# ACRONYMS
acronyms.extend(typF.loc[typF['acronym'] == 1, 'typo'])
len_0 = typF.shape[0]
typF = typF[typF['acronym'] != 1]
len_1 = typF.shape[0]
diff = len_0 - len_1
print("Acronyms removed: " + str(diff))
len_0 = len_1

Number of new slangs: 29
Number of total slangs: 664
Entities removed: 113
Acronyms removed: 10


In [98]:
# From manual:
shorthands = ['Desc', 'Pmt',"Gov't", 'Pmts', 'Xwife', "Int'l", 'Intl', 'Pymt', 'Admin', 'admin', 'Pymts', 'pcm', 'FYI', 'bdrm']
s = typF.loc[typF['shorthand'] == 1, 'typo']
shorthands.extend(s)
print("Number of new shorthands: " + str(len(s)))
print("Number of total shorthands: " + str(len(shorthands)))

Number of new shorthands: 62
Number of total shorthands: 76


In [99]:
print("non-typos: " + str(len(typF.loc[typF['not_typo'] == 1, 'typo'])))
# Remove non-typos - false positives
typF = typF[typF['not_typo'] != 1]
typF = typF[typF['shorthand'] != 1]
typF = typF[typF['slang'] != 1]
# There were some medical typos, so keeping them in there

non-typos: 136


In [100]:
# Purposeful typos
print("Slangs: " + str(len(slangs)))
print("Shorthands: " + str(len(shorthands)))

# Non-typos
print("Acronyms: " + str(len(acronyms)))
print("Entities: " + str(len(entities)))
print("Medical terms: " + str(len(medical_terms)))
print("Medical typos: " + str(len(typF.loc[typF['medical'] == 1, 'typo'])))

print("Remaining typos: " + str(len(typF['typo'])))
typos = typF['typo']

Slangs: 664
Shorthands: 76
Acronyms: 645
Entities: 1891
Medical terms: 64
Medical typos: 28
Remaining typos: 4386


In [101]:
# Unintentional typos:
# Use soundex to see if it's a phoenetic equivalent
# Add a column for phonetic equivalent
# See if typos are phonetically equivalent & get orthographic distance
import soundex

from tqdm import tqdm

phonetic_eq = []
s = soundex.getInstance()

for i, row in tqdm(typF.iterrows()):
    typo = str(row['typo'])
    corr = str(row['fixed'])
    if s.soundex(typo) == s.soundex(corr):
        phonetic_eq.append(1)
    else:
        phonetic_eq.append(0)

4386it [00:00, 6514.45it/s]


In [102]:
print("Percentage of phonetic equivalent: ")
phonetic_eq.count(1)/len(typF['typo'])

Percentage of phonetic equivalent: 


0.7286821705426356

In [103]:
typF['phonetic_eq'] = phonetic_eq

# Orthographic distance:
#import Levenshtein - getting import error
# Source: https://medium.com/@yash_agarwal2/soundex-and-levenshtein-distance-in-python-8b4b56542e9e
def get_levenshtein_distance(word1, word2):
    """
    https://en.wikipedia.org/wiki/Levenshtein_distance
    :param word1:
    :param word2:
    :return:
    """
    word2 = word2.lower()
    word1 = word1.lower()
    matrix = [[0 for x in range(len(word2) + 1)] for x in range(len(word1) + 1)]

    for x in range(len(word1) + 1):
        matrix[x][0] = x
    for y in range(len(word2) + 1):
        matrix[0][y] = y

    for x in range(1, len(word1) + 1):
        for y in range(1, len(word2) + 1):
            if word1[x - 1] == word2[y - 1]:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1],
                    matrix[x][y - 1] + 1
                )
            else:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1] + 1,
                    matrix[x][y - 1] + 1
                )

    return matrix[len(word1)][len(word2)]

levenshtein_dist = []

for i, row in tqdm(typF.iterrows()):
    typo = str(row['typo'])
    corr = str(row['fixed'])
    levenshtein_dist.append(get_levenshtein_distance(typo, corr))

4386it [00:00, 5179.50it/s]


In [104]:
typF['levenshtein_dist'] = levenshtein_dist
typF.index = typF['typo']
typo_dict = dict(zip(typF['typo'], typF['fixed']))
typo_dict['reqarding'] = 'rewarding' #one manual fix
phon_dict = dict(zip(typF['typo'], typF['phonetic_eq']))
lev_dict = dict(zip(typF['typo'], typF['levenshtein_dist']))


In [105]:
"""

# Data source: https://github.com/glutanimate/wordlist-medicalterms-en

with open("medicalwordlist.txt") as f:
    content = f.readlines()
official_medical_words = [x.strip() for x in content] 
official_medical_words.extend(medical_terms)

from tqdm import tqdm

official_medical_words = set(official_medical_words)

n_medical = []
n_slang = []
n_shorthands = []
n_typos = []
n_phonetic_eq = []
avg_lev_dist = []
total_lev_dist = []
n_words = []
n_numbers = []

clean_descs = []

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

# Look for typos
for i, desc in enumerate(tqdm(df['desc_clean'])):
    # Initialize        
    n = 0
    desc_clean = ''
    med = 0
    slang = 0
    shorthand = 0
    typo = 0
    phonetic_eq = 0
    lev_avg = 0
    lev_total = 0
    words = 0
    num = 0

    for word in desc.split(): 
        if word in official_medical_words:
            med += 1
        if word in slangs:
            slang += 1
        if word in shorthands:
            shorthand += 1
        if hasNumbers(word):
            num += 1
        if word in typo_dict.keys():
            typo += 1
            desc_clean += typo_dict[word] # get the correction and add it to the clean text
            desc_clean += " "
            phonetic_eq += phon_dict[word] 
            lev_total += lev_dict[word]
        else:
            desc_clean += word
            desc_clean += " "
                
    clean_descs.append(desc_clean.strip())
    n_numbers.append(num)
    n_typos.append(typo)
    n_medical.append(med)
    n_slang.append(slang)
    n_shorthands.append(shorthand)
    n_phonetic_eq.append(phonetic_eq)
    total_lev_dist.append(lev_total)
    n = len(desc_clean.strip().split())
    n_words.append(n)
    avg_lev_dist.append(lev_total/n)
    
df['n_words'] = n_words
df['tx_n_medical'] = n_medical
df['tx_n_slang'] = n_slang
df['tx_n_shorthands'] = n_shorthands
df['tx_n_typos'] = n_typos
df['tx_n_phonetic_eq'] = n_phonetic_eq
df['tx_avg_lev_dist'] = avg_lev_dist
df['tx_total_lev_dist'] = total_lev_dist
df['tx_n_numbers'] = n_numbers
df['desc_without_typo'] = clean_descs

df.head()

df.to_json("LoanStats_added_features.json")"""

'\n\n# Data source: https://github.com/glutanimate/wordlist-medicalterms-en\n\nwith open("medicalwordlist.txt") as f:\n    content = f.readlines()\nofficial_medical_words = [x.strip() for x in content] \nofficial_medical_words.extend(medical_terms)\n\nfrom tqdm import tqdm\n\nofficial_medical_words = set(official_medical_words)\n\nn_medical = []\nn_slang = []\nn_shorthands = []\nn_typos = []\nn_phonetic_eq = []\navg_lev_dist = []\ntotal_lev_dist = []\nn_words = []\nn_numbers = []\n\nclean_descs = []\n\ndef hasNumbers(inputString):\n    return any(char.isdigit() for char in inputString)\n\n# Look for typos\nfor i, desc in enumerate(tqdm(df[\'desc_clean\'])):\n    # Initialize        \n    n = 0\n    desc_clean = \'\'\n    med = 0\n    slang = 0\n    shorthand = 0\n    typo = 0\n    phonetic_eq = 0\n    lev_avg = 0\n    lev_total = 0\n    words = 0\n    num = 0\n\n    for word in desc.split(): \n        if word in official_medical_words:\n            med += 1\n        if word in slangs:\

In [106]:
df = pd.read_json("LoanStats_added_features.json")
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    #print("{:-<40} {}".format(sentence, str(score)))
    return score
    
sentiment_analyzer_scores(df['desc_without_typo'][1])

{'neg': 0.0, 'neu': 0.948, 'pos': 0.052, 'compound': 0.7003}

In [107]:
""" # Run once
from tqdm import tqdm

neg_list = []
neu_list = []
pos_list = []
compound_list = []

# Get sentiment scores
for i, desc in enumerate(tqdm(df['desc_without_typo'])):
    score = sentiment_analyzer_scores(desc)
    neg_list.append(score['neg'])
    neu_list.append(score['neu'])
    pos_list.append(score['pos'])
    compound_list.append(score['compound'])
    

df['sentiment_neg'] = neg_list
df['sentiment_neu'] = neu_list
df['sentiment_pos'] = pos_list
df['sentiment_compound'] = compound_list

df.to_json("LoanStats_added_features.json") # For future reading

df.head()
"""

' # Run once\nfrom tqdm import tqdm\n\nneg_list = []\nneu_list = []\npos_list = []\ncompound_list = []\n\n# Get sentiment scores\nfor i, desc in enumerate(tqdm(df[\'desc_without_typo\'])):\n    score = sentiment_analyzer_scores(desc)\n    neg_list.append(score[\'neg\'])\n    neu_list.append(score[\'neu\'])\n    pos_list.append(score[\'pos\'])\n    compound_list.append(score[\'compound\'])\n    \n\ndf[\'sentiment_neg\'] = neg_list\ndf[\'sentiment_neu\'] = neu_list\ndf[\'sentiment_pos\'] = pos_list\ndf[\'sentiment_compound\'] = compound_list\n\ndf.to_json("LoanStats_added_features.json") # For future reading\n\ndf.head()\n'

In [108]:
df = pd.read_json("LoanStats_added_features.json")
df.head()

,addr_state,annual_inc,desc,desc_clean,desc_without_typo,emp_length,has_words,home_ownership,id,len_desc,...,tx_n_shorthands,tx_n_slang,tx_n_typos,tx_total_lev_dist,words_desc,zip_code,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
0,AZ,"24,000.0",Borrower added on 12/22/11 > I need to upgra...,I need to upgrade my business technologies.,I need to upgrade my business technologies.,10+ years,1,RENT,0,43,...,0,0,0,0,7,860xx,0.0,1.0,0.0,0.0
1,GA,"30,000.0",Borrower added on 12/22/11 > I plan to use t...,I plan to use this money to finance the motorc...,I plan to use this money to finance the motorc...,< 1 year,1,RENT,1,519,...,0,0,3,4,106,309xx,0.0,0.9430000000000001,0.057,0.7003
10,AZ,"72,000.0",Borrower added on 12/15/11 > I had recived a...,I had recived a loan from Citi Financial about...,I had received a loan from Citi Financial abou...,5 years,1,OWN,10,220,...,0,0,1,1,43,853xx,0.05,0.8270000000000001,0.123,0.4767
100,CA,"25,000.0",Borrower added on 12/14/11 > This loan will ...,This loan will consolidate a current personal ...,This loan will consolidate a current personal ...,2 years,1,RENT,100,192,...,0,0,1,1,33,902xx,0.0,0.9380000000000001,0.062,0.25
1000,NY,"35,000.0",Borrower added on 12/05/11 > I plan to use t...,I plan to use these funds to better manage my ...,I plan to use these funds to better manage my ...,< 1 year,1,RENT,999,304,...,0,0,0,0,55,111xx,0.04,0.647,0.312,0.9412


In [109]:
# Finally, contrast the above features for defaulters vs. non-defaulters

#- % typos / # words for defaulters vs. non-defaulters
df['tx_perc_typo'] = df['tx_n_typos']/df['n_words']
df['tx_perc_phonetic_eq'] = df['tx_n_phonetic_eq']/df['tx_n_typos']
df['tx_n_typos' == 0, 'tx_perc_phonetic_eq'] = 1
print("No typos for: ", round(df['tx_n_typos'].value_counts()[0]/df.shape[0]*100, 2), "%")
print("No slangs for: ", round(df['tx_n_slang'].value_counts()[0]/df.shape[0]*100, 2), "%")

keep = ['loan_status',                 'tx_n_medical',
                         'tx_n_slang',              'tx_n_shorthands',
                         'tx_n_typos',             'tx_n_phonetic_eq',
                    'tx_avg_lev_dist',            'tx_total_lev_dist',
                       'tx_n_numbers',            'desc_without_typo',
                      'sentiment_neg',                'sentiment_neu',
                      'sentiment_pos',           'sentiment_compound',
                       'tx_perc_typo',          'tx_perc_phonetic_eq']
df2 = df[keep]
# Summarise income, loan amount by race
by_default = df2.groupby(["loan_status"]).describe()
by_default

df['loan_status'] = df['loan_status'].map({1: 'Charged Off', 0: 'Fully Paid'})

import warnings
warnings.filterwarnings('ignore') #to hide warnings showing the directory name (for submission anonymity)

# Summarise income, loan amount by race
by_default = df.groupby(["loan_status"]).describe()
by_default

# keep the relevant statistics
keep = [('sentiment_compound', 'count')
        , ('sentiment_compound','mean'), ('sentiment_compound','std'), ('sentiment_compound','50%')
        , ('sentiment_neg','mean'), ('sentiment_neg','std'), ('sentiment_neg','50%')
        , ('sentiment_neu','mean'), ('sentiment_neu','std'), ('sentiment_neu','50%')
        , ('sentiment_pos','mean'), ('sentiment_pos','std'), ('sentiment_pos','50%')]
by_default = by_default[keep]

# flatten multi-index file
by_default.columns = [' '.join(col).strip() for col in by_default.columns.values]
by_default = np.round(by_default, decimals=2)

by_default.columns = ['N', 'Mean: Compound', 'Std dev: Compound', 'Median: Compound'
                      ,'Mean: Negative', 'Std dev: Negative', 'Median: Negative'
                     ,'Mean: Neutral', 'Std dev: Neutral', 'Median: Neutral'
                     ,'Mean: Positive', 'Std dev: Positive', 'Median: Positive' ]

by_default = np.round(by_default, decimals=2)
by_default['N'] = by_default['N'].astype('int')
by_default['N'] = by_default['N'].apply('{:,}'.format)

temp = by_default.unstack().reset_index()
temp.columns = ['Metrics', 'Loan Status', 'Value']
temp = temp.pivot(index='Metrics', columns='Loan Status', values='Value')
temp = temp.reindex(['N', 'Mean: Compound', 'Std dev: Compound', 'Median: Compound'
                      ,'Mean: Negative', 'Std dev: Negative', 'Median: Negative'
                     ,'Mean: Neutral', 'Std dev: Neutral', 'Median: Neutral'
                     ,'Mean: Positive', 'Std dev: Positive', 'Median: Positive'])
temp

latex_output = temp.to_latex()
with open("summary_sentiment.txt", "w") as text_file:
    text_file.write(latex_output)

No typos for:  88.13 %
No slangs for:  97.95 %


In [110]:
import warnings
warnings.filterwarnings('ignore') #to hide warnings showing the directory name (for submission anonymity)

by_default = df.groupby(["loan_status"]).describe()
by_default

#'tx_avg_lev_dist', 'tx_n_medical', 'tx_n_numbers', 'tx_n_phonetic_eq'
#, 'tx_n_shorthands', 'tx_n_slang', 'tx_n_typos', 'tx_perc_phonetic_eq', 'tx_perc_typo', 'tx_total_lev_dist'

# keep the relevant statistics
keep = [('n_words', 'count')
        , ('tx_total_lev_dist', 'mean'), ('tx_total_lev_dist', 'std'),('tx_total_lev_dist', '50%')
        , ('tx_avg_lev_dist','mean'), ('tx_avg_lev_dist','std'), ('tx_avg_lev_dist','50%')
        , ('tx_perc_phonetic_eq','mean'), ('tx_perc_phonetic_eq','std'), ('tx_perc_phonetic_eq','50%')]
by_default = by_default[keep]

# flatten multi-index file
by_default.columns = [' '.join(col).strip() for col in by_default.columns.values]
by_default = np.round(by_default, decimals=2)

by_default.columns = ['N', 'Mean: Total Levenshtein Distance', 'Std dev: Total Levenshtein Distance', 'Median: Total Levenshtein Distance'
                      , 'Mean: Avg Levenshtein Distance', 'Std dev: Avg Levenshtein Distance', 'Median: Avg Levenshtein Distance'
                     ,'Mean: % Phonetic Equivalent', 'Std dev: % Phonetic Equivalent', 'Median: % Phonetic Equivalent']

by_default = np.round(by_default, decimals=2)
by_default['N'] = by_default['N'].astype('int')
by_default['N'] = by_default['N'].apply('{:,}'.format)

by_default

,N,Mean: Total Levenshtein Distance,Std dev: Total Levenshtein Distance,Median: Total Levenshtein Distance,Mean: Avg Levenshtein Distance,Std dev: Avg Levenshtein Distance,Median: Avg Levenshtein Distance,Mean: % Phonetic Equivalent,Std dev: % Phonetic Equivalent,Median: % Phonetic Equivalent
loan_status,,,,,,,,,,
Charged Off,"6,403",0.27,0.94,0.0,0.01,0.02,0.0,0.77,0.39,1.0
Fully Paid,"35,906",0.2,0.72,0.0,0.0,0.03,0.0,0.78,0.38,1.0


In [111]:
temp = by_default.unstack().reset_index()
temp.columns = ['Metrics', 'Loan Status', 'Value']
temp = temp.pivot(index='Metrics', columns='Loan Status', values='Value')
temp = temp.reindex(['N', 'Mean: Total Levenshtein Distance', 'Std dev: Total Levenshtein Distance', 'Median: Total Levenshtein Distance'
                      , 'Mean: Avg Levenshtein Distance', 'Std dev: Avg Levenshtein Distance', 'Median: Avg Levenshtein Distance'
                     ,'Mean: % Phonetic Equivalent', 'Std dev: % Phonetic Equivalent', 'Median: % Phonetic Equivalent'])

latex_output = temp.to_latex()
with open("summary_distance.txt", "w") as text_file:
    text_file.write(latex_output)

In [112]:
import warnings
warnings.filterwarnings('ignore') #to hide warnings showing the directory name (for submission anonymity)

by_default = df.groupby(["loan_status"]).describe()
by_default

#'tx_avg_lev_dist', 'tx_n_medical', 'tx_n_numbers', 'tx_n_phonetic_eq'
#, 'tx_n_shorthands', 'tx_n_slang', 'tx_n_typos', 'tx_perc_phonetic_eq', 'tx_perc_typo', 'tx_total_lev_dist'

# keep the relevant statistics
keep = [('n_words', 'count')
        , ('tx_n_shorthands', 'mean'), ('tx_n_shorthands', 'std'),('tx_n_shorthands', '50%')
        , ('tx_n_typos','mean'), ('tx_n_typos','std'), ('tx_n_typos','50%')
        , ('tx_n_slang','mean'), ('tx_n_slang','std'), ('tx_n_slang','50%')
        , ('tx_n_medical','mean'), ('tx_n_medical','std'), ('tx_n_medical','50%')]
by_default = by_default[keep]

# flatten multi-index file
by_default.columns = [' '.join(col).strip() for col in by_default.columns.values]
by_default = np.round(by_default, decimals=2)

by_default.columns = ['N', 'Mean: # Shorthands', 'Std dev: # Shorthands', 'Median: # Shorthands'
                      , 'Mean: # Typos', 'Std dev: # Typos', 'Median: # Typos'
                     ,'Mean: # Slang', 'Std dev: # Slang', 'Median: # Slang'
                     ,'Mean: # Medical', 'Std dev: # Medical', 'Median: # Medical']

by_default = np.round(by_default, decimals=2)
by_default['N'] = by_default['N'].astype('int')
by_default['N'] = by_default['N'].apply('{:,}'.format)

by_default

,N,Mean: # Shorthands,Std dev: # Shorthands,Median: # Shorthands,Mean: # Typos,Std dev: # Typos,Median: # Typos,Mean: # Slang,Std dev: # Slang,Median: # Slang,Mean: # Medical,Std dev: # Medical,Median: # Medical
loan_status,,,,,,,,,,,,,
Charged Off,"6,403",0.01,0.09,0.0,0.24,0.77,0.0,0.03,0.23,0.0,9.8,15.61,4.0
Fully Paid,"35,906",0.0,0.08,0.0,0.17,0.59,0.0,0.02,0.18,0.0,9.41,14.11,4.0


In [113]:
temp = by_default.unstack().reset_index()
temp.columns = ['Metrics', 'Loan Status', 'Value']
temp = temp.pivot(index='Metrics', columns='Loan Status', values='Value')
temp = temp.reindex(['N', 'Mean: # Shorthands', 'Std dev: # Shorthands', 'Median: # Shorthands'
                      , 'Mean: # Typos', 'Std dev: # Typos', 'Median: # Typos'
                     ,'Mean: # Slang', 'Std dev: # Slang', 'Median: # Slang'
                     ,'Mean: # Medical', 'Std dev: # Medical', 'Median: # Medical'])
temp

latex_output = temp.to_latex()
with open("summary_typos.txt", "w") as text_file:
    text_file.write(latex_output)

## Part 3: Predict default

In [114]:
df['loan_status'] = df['loan_status'].map({'Charged Off':1, 'Fully Paid':0})
# Split into features and labels
y, X = dmatrices("loan_status ~ loan_amnt + annual_inc", data=df, return_type='dataframe') #+ C(home_ownership,Treatment(reference='RENT')) + C(purpose,Treatment(reference='debt_consolidation'))
X.columns = ["Intercept", "Loan amount", "Annual income"] #"Home ownership: mortgage", "Home ownership: none", "Home ownership: other","Home ownership: own", "Purpose: car", "Purpose: credit card", "Purpose: educational", "Purpose: home improvement", "Purpose: house", "Purpose: major purchase", "Purpose: medical", "Purpose: moving", "Purpose: other", "Purpose: renewable energy", "Purpose: small business", "Purpose: vacation", "Purpose: wedding"

In [126]:
# Try scaling
from sklearn.preprocessing import StandardScaler
scaled_X = pd.DataFrame(StandardScaler().fit_transform(X), columns = ["Intercept", "Loan amount", "Annual income"])
scaled_X

,Intercept,Loan amount,Annual income
0,0.0,-0.8221817721079037,-0.7035464817182224
1,0.0,-1.1592795879754982,-0.610041950052741
2,0.0,-0.619923082587347,0.044489771605629266
3,0.0,-0.6603748204914583,-0.6879623931073089
4,0.0,0.39137036501543665,-0.5321215069981731
...,...,...,...
42300,0.0,-0.8221817721079037,0.683437404653086
42301,0.0,0.5228385132037985,-0.31238585758429166
42302,0.0,-1.0109565489937566,0.9795350882604439
42303,0.0,0.5262094913624744,-0.5539392310534521


In [151]:
print("Mean: ")
print(scaled_X.mean())

print("Std: ")
print(scaled_X.std())

Mean: 
Intercept                          0.0
Loan amount     -7.591597707268732e-16
Annual income   1.9124283766341812e-16
dtype: float64
Std: 
Intercept                      0.0
Loan amount     1.0000118191434066
Annual income   1.0000118191435166
dtype: float64


In [132]:
print("Mean: ")
print(X.mean())
print("Std: ")
print(X.std())

Mean: 
Intercept                      1.0
Loan amount     11,097.50147736674
Annual income   69,145.17975889376
dtype: float64
Std: 
Intercept                       0.0
Loan amount      7,416.332679060225
Annual income   64,168.771373845775
dtype: float64


In [152]:
# View the summary
logit = sm.Logit(y, X)
results = logit.fit()
results.summary2()

Optimization terminated successfully.
         Current function value: 0.421451
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.008     
Dependent Variable: loan_status      AIC:              35664.9833
Date:               2020-04-22 14:37 BIC:              35690.9413
No. Observations:   42305            Log-Likelihood:   -17829.   
Df Model:           2                LL-Null:          -17982.   
Df Residuals:       42302            LLR p-value:      7.7995e-67
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-----------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept        -1.6734   0.0297 -56.3143 0.0000 -1.7316 -1.6151
Loan amount       0.0000   0.0000  14.3088 0.0000  0.0000  0.0000
Annual income    -0.0000   0.0000 -13.5691 0.0000 -0.0000 -0.0000
=================================================================

"""

In [160]:
scaled_X = scaled_X.reindex(y.index)

In [162]:
# View the summary
logit = sm.Logit(np.array(y), np.array(scaled_X))
results = logit.fit()
results.summary2()

MissingDataError: exog contains inf or nans

In [51]:
import statsmodels.api as sm
# Split into features and labels

y, X = dmatrices("loan_status ~ loan_amnt + annual_inc + n_words + tx_n_shorthands + tx_n_slang", data=df, return_type='dataframe')

X.head()

,Intercept,loan_amnt,annual_inc,n_words,tx_n_shorthands,tx_n_slang
0,1.0,"5,000.0","24,000.0",7.0,0.0,0.0
1,1.0,"2,500.0","30,000.0",108.0,0.0,0.0
10,1.0,"6,500.0","72,000.0",43.0,0.0,0.0
100,1.0,"6,200.0","25,000.0",33.0,0.0,0.0
1000,1.0,"14,000.0","35,000.0",55.0,0.0,0.0


In [52]:
X.columns = ["Intercept", "Loan amount", "Annual income", "Number of words", "# Shorthands", "# Slang"]
# View the summary
logit = sm.Logit(y, X)
results = logit.fit()
results.summary2()

latex_output = results.summary2().as_latex()
with open("01_model_purposeful_typo_Logit_LaTeX.txt", "w") as text_file:
    text_file.write(latex_output)

Optimization terminated successfully.
         Current function value: 0.421260
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.009     
Dependent Variable: loan_status      AIC:              35654.7853
Date:               2020-04-18 17:31 BIC:              35706.7013
No. Observations:   42305            Log-Likelihood:   -17821.   
Df Model:           5                LL-Null:          -17982.   
Df Residuals:       42299            LLR p-value:      3.6527e-67
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-----------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept        -1.6812   0.0307 -54.7230 0.0000 -1.7414 -1.6210
Loan amount       0.0000   0.0000  14.2440 0.0000  0.0000  0.0000
Annual income    -0.0000   0.0000 -13.5540 0.0000 -0.0000 -0.0000
Number of words  -0.0000   0.0002  -0.0294 0.9765 -0.0004  0.0004
# Shorthands      0.2408   0.1435   1.6780 0.0934 -0.0405  0.5221
# Slang           0.2307   0.0625   3.6880 0.0002  0.1081  0.3532
=================================================================

"""

In [53]:
from math import e
median_income = df['annual_inc'].median()
median_loan = df['loan_amnt'].median()
median_n_words = df['n_words'].median()
print("Median number of words: " + str(median_n_words))

# Function to return the probability, computes e^x/(1+e^x)
coefs = results.params.values
coefs

print("If you have median income/loan/description length with no shorthand or slang, your probability of default is:")
e_value = e**(coefs[0]+(coefs[1]*median_loan)+(coefs[2]*median_income)+(coefs[3]*median_n_words))
print(np.round(e_value/(1+e_value)*100, 2), "%")

print("If you have median income/loan/description length and have a shorthand, your probability of default is:")
e_value = e**(coefs[0]+(coefs[1]*median_loan)+(coefs[2]*median_income)+(coefs[3]*median_n_words)+(coefs[4]))
print(np.round(e_value/(1+e_value)*100, 2), "%")

print("If you have median income/loan/description length and have a slang, your probability of default is:")
e_value = e**(coefs[0]+(coefs[1]*median_loan)+(coefs[2]*median_income)+(coefs[3]*median_n_words)+(coefs[5]))
print(np.round(e_value/(1+e_value)*100, 2), "%")

Median number of words: 22.0
If you have median income/loan/description length with no shorthand or slang, your probability of default is:
14.95 %
If you have median income/loan/description length and have a shorthand, your probability of default is:
18.27 %
If you have median income/loan/description length and have a slang, your probability of default is:
18.12 %
